In [78]:
import blackjack_engine
import random
import numpy as np
import pandas as pd

In [79]:
# from blackjack_api_example.py
def padarray(A, size):
    t = size - len(A)
    return np.pad(A, pad_width=(0, t), mode='constant')

def player_bot(hand):
    """ makes random choice, or makes decisions using neural net etc"""
    # decision making code goes here
    return random.choice(['hit', 'stay']) 

def play_blackjack_game():
    # initialize a game with shuffled deck and hands dealt to dealer and player:
    game = blackjack_engine.BlackjackGame()
    moves = []
    if game.is_finished == False:
        while True: 
            hit_or_stay = player_bot(game.player_hand)
            moves = np.append(moves,hit_or_stay)
            game.player_move(hit_or_stay)
            if game.result == None:
                continue # game is not finished so continue loop
            if game.result in [-1, 0, 1, 1.5]:
                break # game is finished so exit loop
    return game, moves

In [82]:
# implement a training set generation routine: give the API random actions and see how games turn out
def build_training_set(n_pts):
    '''
    state_of_play is an n x m array of values where n is the number of data points, and m is the max number of features in a hand 
    y is an n x 1 array, where n is the number of data points (number of times the player drew a card, basically) 
    '''
    X = []
    y = []
    for pt in np.linspace(0,n_pts,n_pts+1):
        game, moves = play_blackjack_game()
        player_hand = game.player_hand.cards
        dealer_card = game.dealer_hand.cards[0]
        game_result = game.result
        move_sequence = moves
        input_row = {'player_hand_1':player_hand[0],'player_hand_2':player_hand[1],'dealer_card':dealer_card,'game_result':game_result}
        X.append(input_row)
        output_row = padarray(move_sequence,8)
        y.append(output_row)    
    return X, y                      

In [83]:
n_pts = 10
X, y = build_training_set(n_pts)
input_df = pd.DataFrame(X)
output_df = pd.DataFrame(y)

In [88]:
input_df.head(10)

,player_hand_1,player_hand_2,dealer_card,game_result
0,8,11,7,-1
1,9,13,5,1
2,13,10,2,1
3,9,8,1,-1
4,3,11,5,1
5,4,7,6,-1
6,11,13,8,-1
7,10,2,12,-1
8,10,1,13,0
9,11,13,12,0


In [87]:
output_df.head(10)

,0,1,2,3,4,5,6,7
0,hit,0,0,0,0,0,0,0
1,stay,0,0,0,0,0,0,0
2,stay,0,0,0,0,0,0,0
3,stay,0,0,0,0,0,0,0
4,stay,0,0,0,0,0,0,0
5,stay,0,0,0,0,0,0,0
6,hit,0,0,0,0,0,0,0
7,hit,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0
9,stay,0,0,0,0,0,0,0


In [86]:
# train the neural network on the previously generated data
def build_model(X,y):
    input_shape = np.shape(X)
    output_shape = np.shape(y) 
    # use a simple sequential dense model to predict which actions will result in a win
    return model

def train_model(untrained_model,training_data):
    training_data = [X_train, y_train, X_test, y_test]
    
    return trained_network

def use_model_to_make_decision(trained_model,state_of_play):

    decision = trained_model.predict(state_of_play)
    while game_is_still_in_play:
        state_of_play = play_blackjack_hand(state_of_play,decision)
        game_is_still_in_play = check_result(state_of_play)
    return state_of_play
    